In [29]:
class Queue:
    def __init__(self):
        self.queue = []
    
    def put(self,val):
        self.queue.append(val)

    def get(self):
        return self.queue.pop(0)
    
    def empty(self):
        return len(self.queue) == 0
    
    def clear(self):
        self.queue = []
    
    def __str__(self):
        return str(self.queue)

class Grid:
    def __init__(self, width, height):
        self.grid = [[0 for x in range(width)] for y in range(height)]
        self.width = width
        self.height = height
    
    def __str__(self):
        grid_string = ""
        for row in self.grid:
            row_str = ""
            for val in row:
                row_str += str(val)
            grid_string += row_str + "\n"
        return grid_string
     
    """
    Gets the value at the given coordinate
    """
    def get_val(self,x,y):
        try:
            return self.grid[y][x]
        except:
            return None
    
    """
    Get the coordinates which are valid neighbours
    A valid neighbour is a orthogonal coordinate which is an open space
    """
    def get_neighbors(self,xy):
        x,y = xy
        valid = []
        directions = [[1,0],[-1,0],[0,1],[0,-1]]
        for direction in directions:
            if self.get_val(x+direction[0], y+direction[1]) == ".":
                valid.append([x+direction[0], y+direction[1]])
        return valid
        
    """
    Generate the map in accordance to the given algorithm
    """
    def generate_map(self, key):
        for y in range(height):
            for x in range(width):
                self.grid[y][x] = "." if self.__is_open_space(x,y,key) else "#"
    
    """
    If the value at the given grid coordinate is an open space, return True.
    Otherwize return False
    """
    def __is_open_space(self,x,y,key):
        val1 = x*x + 3*x + 2*x*y + y + y*y
        val1 += key
        binary_val = bin(val1)[2:]
        nr_ones = 0
        for x in binary_val:
            if x == "1":
                nr_ones += 1
        if nr_ones % 2 == 0:
            return True
        else:
            return False
        
    def draw_path(self, path, start, goal):
        old_grid = self.grid
        for p in path:
            x = int(p[0])
            y = int(p[1])
            self.grid[y][x] = "O"
            
        self.grid[start[1]][start[0]] = "S"
        self.grid[goal[1]][goal[0]] = "G"
        
        print(self.__str__())
        self.grid = old_grid

### Implement BFS - Breath First Search
Keep track of the frontier! I.e the nodes we want to visit.  

For each node we visit, find its valid neighbors.
If we haven't visited the neighbor before, add it to our frontier queue and mark that we reached that frontier node from the one we're currently on.

In [30]:
class PathFinder:
    def __init__(self, network):
        self.network = network

    def BFS(self, start, goal):
        frontier = Queue()
        frontier.put(start)
        reached_via = {}
        reached_via[str(start)] = None
        goal_reached = False

        while not frontier.empty():
            current = frontier.get()
            neighbors = self.network.get_neighbors(current)
            for neighbor in neighbors:
                if str(neighbor) not in reached_via:
                    frontier.put(neighbor)
                    reached_via[str(neighbor)] = current
                    
                    if neighbor == goal:
                        frontier.clear()
                        goal_reached = True
                        break
        
        if not goal_reached:
            return False

        # Backtrack to get shortest path
        prev_node = goal
        shortest_path = []
        while True:
            shortest_path.insert(0,prev_node)
            prev_node = reached_via[str(prev_node)]
            if prev_node == None:
                shortest_path = shortest_path[1:]
                break

        return shortest_path
    
    
    def get_all_reachable(self, maxDist):
        frontier = Queue()
        frontier.put(start)
        reached_via = {}
        reached_via[str(start)] = None

        while not frontier.empty():
            current = frontier.get()
            neighbors = self.network.get_neighbors(current)
            for neighbor in neighbors:
                if str(neighbor) not in reached_via:
                    reached_via[str(neighbor)] = current

                    # Do not put neighbour in frontier if the step to there would exceed maxDist if backtracked to start position
                    # Backtrack to get shortest path
                    prev_node = goal
                    shortest_path = []
                    while True:
                        shortest_path.insert(0,prev_node)
                        prev_node = reached_via[str(prev_node)]
                        if prev_node == None:
                            shortest_path = shortest_path[1:]
                            break
                    
                    frontier.put(neighbor)
                    
        return shortest_path

In [32]:
width = 100
height = 60
grid = Grid(width, height)
grid.generate_map(1352)

pathfinder = PathFinder(grid)

shortest_path = pathfinder.BFS([1,1], [31,39])
grid.draw_path(shortest_path, [1,1],[31,39])

print(f"Steps taken: {len(shortest_path)} st")

.##.##............###.....#####.#####....#..##..#.###....#....#......#..#.##.#....##.###.###..##.##.
#S######.##.####...#.##....#..#.#..#.##.####.##....####.###.#.##.##.###......###........#...#..##.#.
.OOO##.#..#.##.#.#.######..#.##..#.####..#.#######..###..##..#.##.#.##########.#####..#.#.#.......##
##.OOO###......#..#..#..####.###..#..#.#.##..#.......###...#....#....##...#......#####..#..##.###.##
.####O####..###.#.##.#.#...#..#.#.##.##......#..####..#.######.#####....#...##.......#.###..#.###...
.....OO######.###..###..##.#..###..##.#..##.####...#..##....##.#...#.###.######.######.#.##....#####
..##.#OO##..#.......###.#..###......#.#####..#.#...#...#..#....#...#...###....#.##..##.######...##.#
##.#...OOOO.##.##.#.....#....#.##.#.###..#.#.####..##.###########..##.#....##.....#..##..#..##.....#
.##.######O#.#.##...###.##...##.#..#..##.##...#######.....##..#######.#########.#.....##.#.#.#.#####
#.##....##OO##.#.###..#.###.#.##.#..#..##.#.....#..#####....#..##..##...#..##.#..##.#..###.